# Genetic algorithms application

The population of individuals are maintained within Search Space. Each individual represents a solution in search space for given problem. Each individual is coded as a finite length vector of components - Chromosome. These variable components are analogous to Genes or Genome. Thus a chromosome (individual) is composed of several genes (variable components).

A Fitness Score is given to each individual which shows the ability of an individual to “compete”. The individual having optimal fitness score (or near optimal) are sought.

The GAs maintains the Population of n individuals (chromosome/solutions) along with their fitness scores.The individuals having better fitness scores are given more chance to reproduce than others. The individuals with better fitness scores are selected who mate and produce better offspring by combining chromosomes of parents.

Each new generation has on average more “better genes” than the individual (solution) of previous generations. Thus each new generations have better “partial solutions” than previous generations.

The Genetic Algorithm evolves the generation using following operators:

1) Selection Operator: The idea is to give preference to the individuals with good fitness scores and allow them to pass their genes to successive generations.

2) Crossover Operator: This represents mating between individuals. Two individuals are selected using selection operator and crossover sites are chosen randomly. Then the genes at these crossover sites are exchanged thus creating a completely new individual (offspring).

3) Mutation Operator: The key idea is to insert random genes in offspring to maintain the diversity in the population to avoid premature convergence.

In [3]:
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt
from skfuzzy import control as ctrl
import pandas as pd
from skimage.metrics import structural_similarity as ssim
import imageio
from datetime import datetime

In [4]:
# ... and initialize parameters

n_objects = 20
max_weight = 3

n_population = 100

mutation_rate = 0.3

In [5]:
# Generating a list of weights and values representing each item
weight_value = [[x,y] for x,y in zip(np.random.randint(0,10,n_objects)/10,np.random.randint(0,100,n_objects))]
object_list = np.array(['Water', 'Food', 'Pants', 'Socks', 'Boots', 'Shirts', 'Coat', 'Blanket', 'Laptop', 'Sleeping Bag', \
                        'Cellphone', 'Book', 'Gloves', 'Towel', 'Sunscream', 'Glasses', 'Fork', 'Knife', 'Matches', 'Umbrella'])
objects_dict = { x:y for x,y in zip(object_list,weight_value)}

def get_current_weight_value(objects_list, objects_dict):
    weight = 0
    value = 0
    for o in objects_list:
        o = objects_dict[o]
        weight += o[0]
        value += o[1]
    return weight, value


def s_print(obj_dict):
    for item in set(obj_dict):
      print(item)

objects_dict

{'Water': [0.2, 3],
 'Food': [0.9, 76],
 'Pants': [0.3, 24],
 'Socks': [0.4, 39],
 'Boots': [0.5, 33],
 'Shirts': [0.2, 33],
 'Coat': [0.5, 22],
 'Blanket': [0.5, 49],
 'Laptop': [0.5, 8],
 'Sleeping Bag': [0.7, 58],
 'Cellphone': [0.0, 7],
 'Book': [0.4, 19],
 'Gloves': [0.0, 61],
 'Towel': [0.1, 82],
 'Sunscream': [0.9, 74],
 'Glasses': [0.2, 39],
 'Fork': [0.6, 65],
 'Knife': [0.4, 62],
 'Matches': [0.8, 31],
 'Umbrella': [0.6, 15]}

### Create the first population set

We randomly shuffle the items N times where N=population_size

In [6]:
# First step: Create the first population set
def fit_in_knapsack(objects_list, max_weight, objects_dict):
    r = []
    for o in objects_list:
        r.append(o)
        # Check if count(item) > 1
        for item in r:
          flag = r.count(item) > 1
        weight, value = get_current_weight_value(r, objects_dict)
        if (weight > max_weight) or flag:
            r.pop()
            return r
    return r

def genesis(object_list, n_population, max_weight, objects_dict):

    population_set = []
    for i in range(n_population):
        #Randomly generating a new solution
        sol_i = object_list[np.random.choice(list(range(n_objects)), n_objects, replace=False)]
        sol_i = fit_in_knapsack(sol_i, max_weight, objects_dict)
        population_set.append(sol_i)
    return np.array(population_set, dtype=object)

population_set = genesis(object_list, n_population, max_weight, objects_dict)
population_set

array([list(['Umbrella', 'Sleeping Bag', 'Boots', 'Water', 'Glasses', 'Fork']),
       list(['Umbrella', 'Glasses', 'Sleeping Bag', 'Knife', 'Fork', 'Socks', 'Cellphone']),
       list(['Blanket', 'Glasses', 'Book', 'Towel', 'Socks', 'Fork', 'Sleeping Bag']),
       list(['Sleeping Bag', 'Laptop', 'Book', 'Cellphone', 'Knife', 'Fork', 'Water']),
       list(['Pants', 'Towel', 'Knife', 'Fork', 'Food', 'Glasses']),
       list(['Gloves', 'Fork', 'Shirts', 'Glasses', 'Pants', 'Cellphone', 'Coat', 'Socks', 'Blanket']),
       list(['Matches', 'Water', 'Laptop', 'Cellphone', 'Pants', 'Coat', 'Shirts']),
       list(['Glasses', 'Sunscream', 'Knife', 'Boots', 'Book', 'Water']),
       list(['Cellphone', 'Food', 'Sunscream', 'Gloves', 'Water', 'Blanket', 'Book']),
       list(['Book', 'Boots', 'Knife', 'Socks', 'Food']),
       list(['Food', 'Coat', 'Towel', 'Boots', 'Sleeping Bag']),
       list(['Knife', 'Coat', 'Laptop', 'Boots', 'Book']),
       list(['Book', 'Umbrella', 'Sunscream', 'Cell

### Selection

From each generation, a few individuals are selected based on their fitness scores. These fitness scores are calculated for each individual based on an objective function. This function is based on the problem statement for which the Genetic Algorithm is being used and indicates an individual’s quality.

This fitness score of an individual is an indication of how suitable the respective candidate is for the problem. For each successive generation, a portion of the current population with relatively higher fitness scores is allowed to move to the next iteration. Various schemes such as tournament selection, rank-based selection, roulette wheel selection, elitism selection, etc may be used in the selection stage.

Fitness Function: The fitness function that we use here takes input from the chromosome sequence and the values of the individual items. It then returns the profit by calculating the dot product of the two input vectors.

Parent Selection: For this, we may use the approach of deterministic tournament selection. In this method, we randomly select two individuals from the population and choose the one with higher fitness. We may observe cases where both individuals have low fitness scores, but they still might contribute to choosing the correct genetic sequence. This type of parent selection results in 50% of the population moving to the next stage. To increase the preference for higher fitness, we may include more than two individuals for consideration in the tournament.

### Evaluate solutions fitness

The solutions are defined so that the first element on the list is the first item to take, then the second, etc. The fitness function needs to compute the total weight of the backpack.

In [7]:
def get_all_fitnes(population_set, objects_dict):
    fitnes_list = np.zeros(n_population)

    #Looping over all solutions computing the fitness for each solution
    for i in  range(n_population):
        _, fitnes_list[i] = get_current_weight_value(population_set[i], objects_dict)

    return fitnes_list

fitnes_list = get_all_fitnes(population_set,objects_dict)
fitnes_list

array([213., 285., 351., 222., 348., 339., 128., 230., 289., 229., 271.,
       144., 210., 250., 288., 158., 308., 239., 368., 246., 223., 350.,
       268., 122., 185., 261., 207., 225., 258., 227., 322., 387., 312.,
       213., 375., 249., 243., 192., 323., 240., 162., 316., 320., 231.,
       314., 208., 281., 333., 271., 252., 298., 259., 218., 243., 247.,
       206., 233., 183., 251., 245., 316., 180., 243., 171., 135., 263.,
       194., 224., 286., 323., 303., 246., 256., 214., 344., 126., 336.,
       103., 276., 291., 296., 141., 127., 169., 226., 297., 368., 239.,
       284., 183., 221., 203., 290., 293., 284., 307., 339., 193., 156.,
       238.])

### Progenitors selection

We will select a new set of progenitors using the Roulette Wheel Selection. Generates a list of progenitor pairs where N = len(population_set) but at each position there are two solutions to merge

In [8]:
def progenitor_selection(population_set,fitnes_list):
    total_fit = fitnes_list.sum()
    prob_list = fitnes_list/total_fit

    #Notice there is not (of False) a chance that a progenitor. mates with oneself
    progenitor_list_a = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=False)
    progenitor_list_b = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=False)

    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]


    return np.array([progenitor_list_a,progenitor_list_b])


progenitor_list = progenitor_selection(population_set,fitnes_list)
#progenitor_list[0][2]

### Recombination

This is an essential phase of the Algorithm and involves the creation of a new generation from the current population of the selected individuals. Each individual in the new generation is a result of a crossover operation on a pair of “parent” solutions from the pool of selected individuals.

The new individual created contains traits from both parents based on the Crossover scheme used. The crossover operation is applied over selected individuals using one of the schemes mentioned previously. This results in a completely new population (also called as Children Population).

For each pair of parents we'll generate an offspring pair. Since we cannot repeat items what we'll do is copy a random chunk from one progenitor and fill the blanks with the other progenitor.

In [9]:
def mate_progenitors(prog_a, prog_b, max_weight, objects_dict):
    offspring = []

    for i in zip(prog_a, prog_b):
        offspring.extend(i)
        offspring = list(dict.fromkeys(offspring)) #Removing duplicates
        offspring = fit_in_knapsack(offspring, max_weight, objects_dict)

    return offspring



def mate_population(progenitor_list, max_weight, objects_dict):
    new_population_set = []
    for i in range(progenitor_list.shape[1]):
        prog_a, prog_b = progenitor_list[0][i], progenitor_list[1][i]
        offspring = mate_progenitors(prog_a, prog_b, max_weight, objects_dict)
        new_population_set.append(offspring)

    return new_population_set

new_population_set = mate_population(progenitor_list, max_weight, objects_dict)
new_population_set[0]

['Gloves',
 'Socks',
 'Sunscream',
 'Cellphone',
 'Fork',
 'Blanket',
 'Water',
 'Shirts']

### Mutation

This operation is used to create more genetic diversity in a generation. It alters one or more values from the chromosome, resulting in a completely new genome. This often helps avoid premature convergence.

This is analogous to biological mutation and alters the chromosomes of the children using one or more schemes such as bit-flip mutation, swap mutation, scramble mutation, etc. In simple terms, mutation may be defined as a small random tweak in the chromosomes, to get a new solution.

Now for each element of the new population we add a random chance of swapping

In [10]:
def mutate_offspring(offspring, max_weight, object_list, objects_dict):
    for mutation_number in range(int(len(offspring)*mutation_rate)):

        a = np.random.randint(0,len(object_list))
        b = np.random.randint(0,len(offspring))

        offspring.insert(b, object_list[a])

    offspring = fit_in_knapsack(offspring, max_weight, objects_dict)

    return offspring


def mutate_population(new_population_set, max_weight, object_list, objects_dict):
    mutated_pop = []
    for offspring in new_population_set:
        mutated_pop.append(mutate_offspring(offspring, max_weight, object_list, objects_dict))
    return mutated_pop

mutated_pop = mutate_population(new_population_set, max_weight,object_list, objects_dict)
mutated_pop[0]

['Gloves', 'Socks', 'Sunscream', 'Cellphone', 'Coat', 'Fork', 'Blanket']

### Termination

The loop is terminated when either of the following conditions is met:

- A solution is obtained that satisfies a Threshold fitness.
- The maximum number of generations specified has been processed.
- The allocated computation budget is consumed.
- The best fitness for the new generation does not show any significant improvement and seems to have converged at a point.

To select the stopping criteria we'll need to create a loop to stop first. Then we'll set it to loop at 1000 iterations.

In [11]:
best_solution = [-1,-np.inf,np.array([], dtype=object)]
for i in range(10000):
    if i%100==0: print(i, fitnes_list.min(), fitnes_list.mean(), datetime.now().strftime("%d/%m/%y %H:%M"))
    fitnes_list = get_all_fitnes(mutated_pop,objects_dict)

    #Saving the best solution
    if fitnes_list.max() > best_solution[1]:
        best_solution[0] = i
        best_solution[1] = fitnes_list.max()
        best_solution[2] = np.array(mutated_pop, dtype=object)[fitnes_list.max() == fitnes_list]

    progenitor_list = progenitor_selection(population_set,fitnes_list)
    new_population_set = mate_population(progenitor_list, max_weight, objects_dict)

    mutated_pop = mutate_population(new_population_set, max_weight,object_list, objects_dict)

0 103.0 250.53 02/02/25 19:58
100 7.0 214.91 02/02/25 19:58
200 22.0 207.84 02/02/25 19:58
300 22.0 209.1 02/02/25 19:58
400 22.0 222.94 02/02/25 19:58
500 41.0 215.83 02/02/25 19:58
600 25.0 209.17 02/02/25 19:58
700 15.0 212.76 02/02/25 19:58
800 8.0 220.72 02/02/25 19:58
900 22.0 216.73 02/02/25 19:58
1000 48.0 203.6 02/02/25 19:58
1100 15.0 215.51 02/02/25 19:58
1200 7.0 212.93 02/02/25 19:58
1300 36.0 209.45 02/02/25 19:58
1400 15.0 221.65 02/02/25 19:58
1500 36.0 209.68 02/02/25 19:58
1600 15.0 197.65 02/02/25 19:58
1700 37.0 218.97 02/02/25 19:58
1800 8.0 212.69 02/02/25 19:58
1900 41.0 215.8 02/02/25 19:58
2000 29.0 225.18 02/02/25 19:58
2100 19.0 212.45 02/02/25 19:58
2200 15.0 219.01 02/02/25 19:58
2300 39.0 219.04 02/02/25 19:58
2400 55.0 220.97 02/02/25 19:59
2500 24.0 210.57 02/02/25 19:59
2600 27.0 206.41 02/02/25 19:59
2700 36.0 206.71 02/02/25 19:59
2800 22.0 216.23 02/02/25 19:59
2900 7.0 201.42 02/02/25 19:59
3000 49.0 224.32 02/02/25 19:59
3100 31.0 215.91 02/02/25 1

In [20]:
get_current_weight_value(best_solution[2][0], objects_dict)
print(best_solution[2][0])

['Towel', 'Food', 'Shirts', 'Knife', 'Gloves', 'Sleeping Bag']
